In [207]:
import os
os.getcwd()

'/'

In [208]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic
import pandas as pd
import s3fs

In [209]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [210]:
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
)

In [211]:
bucket = 'projet-hackathon-un-2022'
data = 'AIS/arrival_counts_april_19.csv'

arrivals_19 = pd.read_csv(
    fs.open(f'{bucket}/{data}',
            mode='rb')
).iloc[:, 1:]

In [212]:
test = indic.find_countries_aggregate_table(arrivals_19)

In [219]:
def save_country_aggs_arrivals(df_counts, save_path):
    cargos = ["General Cargo", "Bulk Dry",  "Container", 
              "Other Bulk Dry", "Refrigerated Cargo",
              "Other Dry Cargo", "Bulk Dry / Oil",
              "Self Discharging Bulk Dry"]
    df_agg = df_counts[df_counts['ShipTypeLevel3'].isin(cargos)]
    df_agg = df_agg.groupby(['country', 'country_origin'])[['count', 'GrossTonnage', 'NetTonnage']].sum()
    df_agg.reset_index()

    df_agg.to_csv(
        fs.open(f'{bucket}/{save_path}',
                mode='w')
    )

In [218]:
save_country_aggs_arrivals(test, 'AIS/arrival_aggs_april_19.csv')

In [220]:
data = 'AIS/arrival_counts_april_22.csv'

arrivals_22 = pd.read_csv(
    fs.open(f'{bucket}/{data}',
            mode='rb')
).iloc[:, 1:]

test = indic.find_countries_aggregate_table(arrivals_22)
save_country_aggs_arrivals(test, 'AIS/arrival_aggs_april_22.csv')

In [221]:
def save_country_aggs_departures(df_counts, save_path):
    cargos = ["General Cargo", "Bulk Dry",  "Container", 
              "Other Bulk Dry", "Refrigerated Cargo",
              "Other Dry Cargo", "Bulk Dry / Oil",
              "Self Discharging Bulk Dry"]
    df_agg = df_counts[df_counts['ShipTypeLevel3'].isin(cargos)]
    df_agg = df_agg.groupby(['country', 'country_destination'])[['count', 'GrossTonnage', 'NetTonnage']].sum()
    df_agg.reset_index()

    df_agg.to_csv(
        fs.open(f'{bucket}/{save_path}',
                mode='w')
    )

In [224]:
data = 'AIS/departure_counts_april_19.csv'

departure_19 = pd.read_csv(
    fs.open(f'{bucket}/{data}',
            mode='rb')
).iloc[:, 1:]

test = indic.find_countries_aggregate_table(departure_19, False)
save_country_aggs_departures(test, 'AIS/departure_aggs_april_19.csv')

In [225]:
data = 'AIS/departure_counts_april_22.csv'

departure_22 = pd.read_csv(
    fs.open(f'{bucket}/{data}',
            mode='rb')
).iloc[:, 1:]

test = indic.find_countries_aggregate_table(departure_22, False)
save_country_aggs_departures(test, 'AIS/departure_aggs_april_22.csv')